# NYPD Arrest Data (Historic):

a. To get the required columns, use this module:


1.   get_area_of_interest(df_spark, interested_columns)


b. Preprocessing pipeline: Pass your data through these functions. (if your columns fall in those categories)

1.   valid_date_check(date)
2.   valid_time_check(time)
3.   reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index)
4.   refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None)
5.   refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None)
6.   refine_precinct_jur(df_spark, precinct=None, Jur_code=None)



In [1]:
!pip install pyspark
!pip install openclean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.8/260.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 36.8 MB/s eta 0:00:00
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=155030 sha256=b59fc86c7bb20731c64335f66b6e33917544b56272526ed9f617b520986

# New Section

In [2]:
#importing packages required
from pyspark import SparkContext, SparkConf
import os
import requests
from six.moves import urllib
import sys
import pandas as pd
import matplotlib
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler
from openclean.data.source.socrata import Socrata
from openclean.pipeline import stream
from openclean.function.eval.datatype import IsDatetime
import datetime
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType

In [3]:
from geopy.geocoders import ArcGIS
geocoder=ArcGIS()
#example:
geocoder.reverse('40.61157006600007, -73.74736517199995')

Location(808 Redfern Ave, Far Rockaway, New York 11691, USA, (40.611614718328, -73.747382377557, 0.0))

In [4]:
import os
import urllib.request
from pyspark import SparkContext
from pyspark.sql import SparkSession

# NYC Open Data API endpoint for NYPD Arrests (Historic)
fn_src = 'https://data.cityofnewyork.us/resource/uip8-fykc.csv?$limit=10000'
fn_dst = '/content/NYPD_Arrests_Data_Historic_10k.csv'

# Download only if not already present
if os.path.isfile(fn_dst):
    print('File has already been downloaded:', fn_dst)
else:
    print('Fetching file. This may take a while...', fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File downloaded to:', fn_dst)

Fetching file. This may take a while... /content/NYPD_Arrests_Data_Historic_10k.csv
File downloaded to: /content/NYPD_Arrests_Data_Historic_10k.csv


In [5]:
# Create Spark session
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Load into Spark DataFrame
df_spark = spark.read.option("header", True).csv(fn_dst, inferSchema=True)

# Print basic info
print("Number of rows:", df_spark.count())
df_spark.printSchema()
df_spark.show(5)

Number of rows: 10000
root
 |-- arrest_key: integer (nullable = true)
 |-- arrest_date: timestamp (nullable = true)
 |-- pd_cd: integer (nullable = true)
 |-- pd_desc: string (nullable = true)
 |-- ky_cd: integer (nullable = true)
 |-- ofns_desc: string (nullable = true)
 |-- law_code: string (nullable = true)
 |-- law_cat_cd: string (nullable = true)
 |-- arrest_boro: string (nullable = true)
 |-- arrest_precinct: integer (nullable = true)
 |-- jurisdiction_code: integer (nullable = true)
 |-- age_group: string (nullable = true)
 |-- perp_sex: string (nullable = true)
 |-- perp_race: string (nullable = true)
 |-- x_coord_cd: integer (nullable = true)
 |-- y_coord_cd: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- geocoded_column: string (nullable = true)

+----------+-------------------+-----+--------------------+-----+--------------------+----------+----------+-----------+---------------+-----------------+---------+----

## Data Profiling

In [ ]:
#Using openclean for finding anomalies
from openclean.pipeline import stream
ds = stream(fn_dst)

In [ ]:
#Creating profile of our dataset
from openclean.profiling.column import DefaultColumnProfiler
profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [ ]:
#CMPLNT_NUM, RPT_DT, KY_CD, LAW_CAT_CD have no null values
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,5153369,0,5153369,1.000000e+00,22.297084
ARREST_DATE,5153369,0,5479,1.063188e-03,12.318990
PD_CD,5153369,284,334,6.481554e-05,5.583088
PD_DESC,5153369,9066,422,8.203249e-05,5.761536
KY_CD,5153369,9066,75,1.457923e-05,4.674493
OFNS_DESC,5153369,9066,87,1.691191e-05,4.464132
LAW_CODE,5153369,196,2440,4.734947e-04,6.468133
LAW_CAT_CD,5153369,18861,4,7.790425e-07,1.178222
ARREST_BORO,5153369,8,5,9.702406e-07,2.133827
ARREST_PRECINCT,5153369,0,78,1.513573e-05,6.062884


In [ ]:
# Print the most frequent values in column 'OFNS_DESC'

profiles.column('OFNS_DESC').get('topValues')

[('DANGEROUS DRUGS', 1088738),
 ('ASSAULT 3 & RELATED OFFENSES', 515608),
 ('OTHER OFFENSES RELATED TO THEFT', 304191),
 ('OTHER STATE LAWS', 241509),
 ('PETIT LARCENY', 221393),
 ('FELONY ASSAULT', 210233),
 ('VEHICLE AND TRAFFIC LAWS', 204881),
 ('CRIMINAL TRESPASS', 200983),
 ('DANGEROUS WEAPONS', 197557),
 ('MISCELLANEOUS PENAL LAW', 185927)]

## Create a sample dataset

In [ ]:
df_spark=df_spark.sample(withReplacement=False, fraction=0.0003)

In [ ]:
df_spark.printSchema()

root
 |-- ARREST_KEY: integer (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_PRECINCT: integer (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- X_COORD_CD: double (nullable = true)
 |-- Y_COORD_CD: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



## Relevant Matching Columns:

1. ARREST_DATE
2. ARREST_PRECINCT
3. JURISDICTION_CODE
4. ARREST_BORO
5. Latitude
6. Longitude
7. Lon_Lat
8. PERP_SEX
9. PERP_RACE
10. LAW_CAT_CD
11. AGE_GROUP


In [ ]:
#Check if there are redundant columns for latitude and longitude
print("Lon Lat column samples: ")
print(df_spark.select("Lon_Lat").show(10))
print("Latitude column samples: ")
print(df_spark.select("Latitude").show(10))
print("Longitude column samples: ")
print(df_spark.select("Longitude").show(10))

Lon Lat column samples: 
+--------------------+
|             Lon_Lat|
+--------------------+
|POINT (-73.903320...|
|POINT (-73.734760...|
|POINT (-73.936252...|
|POINT (-73.827329...|
|POINT (-73.944004...|
|POINT (-73.900591...|
|POINT (-73.831129...|
|POINT (-73.900388...|
|POINT (-73.983941...|
|POINT (-73.960854...|
+--------------------+
only showing top 10 rows

None
Latitude column samples: 
+------------------+
|          Latitude|
+------------------+
| 40.88311992000007|
|40.726293088000034|
|40.840783858000066|
|40.762043893000055|
|40.819991180000045|
| 40.85358740100003|
| 40.69843969400006|
| 40.84413994500005|
| 40.71567057300007|
| 40.59401905900006|
+------------------+
only showing top 10 rows

None
Longitude column samples: 
+------------------+
|         Longitude|
+------------------+
|-73.90332096899994|
|-73.73476084899994|
|-73.93625260699997|
|-73.82732958099996|
|-73.94400425099997|
|-73.90059135599995|
|-73.83112953899996|
|-73.90038861799997|
|-73.98394174

We saw that the coordinates are redundant. So, we will remove the Latitude and Longitude columns

In [ ]:
df_spark=df_spark.drop("Lon_Lat")


# Replacing Missing AND Null values of race, sex and age group of victim and suspect with "UNKNOWN"


In [ ]:
df_spark = df_spark.na.fill("U",subset=["PERP_SEX"])

In [ ]:
df_spark = df_spark.na.fill("UNKNOWN",subset=["PERP_RACE"])

In [ ]:
df_spark = df_spark.na.fill("UNKNOWN",subset=["AGE_GROUP"])

In [ ]:
df_spark.count()

1553

# Cleaning the columns associated with the Level of offence-
1. LAW_CAT_CD

In [ ]:
df_tempo=df_spark.rdd

In [ ]:
#offense should belong to one of the categories- 'FELONY',"MISDEMEANOR","VIOLATION"
def level_of_offence_check(offence):
    if offence=="" or offence==" " or offence==None:
        return False
    else:
        crimes_list=['F',"M","V"]
        if offence not in crimes_list:
            return False
        else :
            return True

In [ ]:
#Filtering offence
offense_filtered_df=df_tempo.map(lambda x:(x, level_of_offence_check(x[7]))).filter(lambda x: x[1]==True)
temp=offense_filtered_df.map(lambda x: x[0])

In [ ]:
#Count of filtered or cleaned data available
temp.count()

1537

# Cleaning the columns associated with Perpetrator's age group-

In [ ]:
#Checking if age is not negative or greater than 100
def to_check_age(age):
    if age != None and age != " ":
      try:
        age_list = ["25-44", "18-24", "45-64", "<18", "65+","UNKNOWN"]
        if age not in age_list:
          return False
        else:
          return True
      except:
        return True
    return True

In [ ]:
#Filtering invalid age of suspect
suspect_age_filtered_df=temp.map(lambda x:(x, to_check_age(x[11]))).filter(lambda x: x[1]==True)
temp=suspect_age_filtered_df.map(lambda x: x[0])

In [ ]:
#Count of filtered or cleaned data available
temp.count()

1537

#6. Reverse Geocoding the boroughs using latitudes and longitudes.

1. First we will remove the rows where latitude, longitude and boroughs are null. (around 450 tuples removed)
2. Then, where the boroughs are empty, take the latitude and longitude value and reverse geocode it using the module "reverseGeocoder".
3. Impute the borough name retrived in the empty space.

In [ ]:
df_temp_boro_clean=df_spark.filter((df_spark.Latitude.isNotNull()) & (df_spark.Longitude.isNotNull()))

In [ ]:
boro_cleaner=df_temp_boro_clean.filter(df_temp_boro_clean.ARREST_BORO.isNull())

In [ ]:
boro_cleaner.count()

0


# Cleaning the columns associated with the valid borough name-
1. BORO_NM

In [ ]:
#Complaint should be registered in a valid NYC borough
def valid_borough_check(borough):
    if borough==None or borough=="" or borough==" ":
        return False
    else:
        borough_names=["M",'B',"K","Q","S"]
        if borough not in borough_names:
            return False
        else :
            return True

In [ ]:
borough_filtered_df=temp.map(lambda x:(x, valid_borough_check(x[8]))).filter(lambda x: x[1]==True)
temp=borough_filtered_df.map(lambda x: x[0])

In [ ]:
#Count of filtered or cleaned data available
temp.count()

1537

# Cleaning the columns associated with the valid Latitude and Longitude-

1. Lat_Lon

In [ ]:
#Checking the validity of coordinates
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y

#Bounding Box Lat details for NYC
lat_max=40.917577
lat_min=40.477399
long_max=-73.700009
long_min=-74.25909

#Function to check validity of Lat values
def to_check_long(x):
  try:
    if x== "" or x== " "or x==None:
      return False
    elif (long_min<x) and (x<long_max):
      return True
    else:
      return False
  except ValueError:
    return False

#Function to check validity of Lat values
def to_check_lat(x):
  try:
    if x== "" or x== " "or x==None:
      return False
    elif (lat_min<x) and (x<lat_max):
      return True
    else:
      return False
  except ValueError:
    return False

In [ ]:
#Filtering the latitude and longitude that is invalid or is outside NYC
latlong_filtered_df=temp.map(lambda x:(x, to_check_lat(x[16]))).filter(lambda x: x[1]==True)
temp=latlong_filtered_df.map(lambda x: x[0])
latlong_filtered_df=temp.map(lambda x:(x, to_check_long(x[17]))).filter(lambda x: x[1]==True)
temp=latlong_filtered_df.map(lambda x: x[0])

In [ ]:
#Count of filtered or cleaned data available
temp.count()

1534

In [ ]:
temp.top(100)

[Row(ARREST_KEY=222321641, ARREST_DATE='12/28/2020', PD_CD=244, PD_DESC='BURGLARY,UNCLASSIFIED,UNKNOWN', KY_CD=107, OFNS_DESC='BURGLARY', LAW_CODE='PL 1402000', LAW_CAT_CD='F', ARREST_BORO='B', ARREST_PRECINCT=48, JURISDICTION_CODE=0, AGE_GROUP='45-64', PERP_SEX='M', PERP_RACE='WHITE HISPANIC', X_COORD_CD=1011811.0, Y_COORD_CD=246833.0, Latitude=40.844139945000045, Longitude=-73.90038861799998),
 Row(ARREST_KEY=222043404, ARREST_DATE='12/20/2020', PD_CD=101, PD_DESC='ASSAULT 3', KY_CD=344, OFNS_DESC='ASSAULT 3 & RELATED OFFENSES', LAW_CODE='PL 1200001', LAW_CAT_CD='M', ARREST_BORO='M', ARREST_PRECINCT=17, JURISDICTION_CODE=0, AGE_GROUP='25-44', PERP_SEX='M', PERP_RACE='WHITE HISPANIC', X_COORD_CD=991329.0, Y_COORD_CD=212802.0, Latitude=40.75077427200006, Longitude=-73.97445191099996),
 Row(ARREST_KEY=221927669, ARREST_DATE='12/16/2020', PD_CD=339, PD_DESC='LARCENY,PETIT FROM OPEN AREAS,', KY_CD=341, OFNS_DESC='PETIT LARCENY', LAW_CODE='PL 1552500', LAW_CAT_CD='M', ARREST_BORO='B', ARRE

Earlier, precision and recall were both zero. Because borough names were abbreviated
True Positive = 0
selected elements = 1553
Relevant elements = 1534

Later, when the technique was modified to handle abbreviated borough names
True Positive = 1534
selected elements = 1553
Relevant elements = 1534

precision= 1534/1553
recall = 1534/1534